## Populate MOF with given molecule and initialize LAMMPS
- Read MOF xyz file
- Get size of the MOF
- Center MOF to size/2
- Add molecule
- Run packmol
- Convert xyz to cif

In [1]:
import os, shutil
from angstrom import Molecule
from packmol import Packmol

In [6]:
cells = {'BEDYEQ': [17.1430, 41.9680, 41.9680, 120.0, 90.0, 90.0],
         'IRMOF-1': [25.8320, 25.8320, 25.8320, 90.0, 90.0, 90.0],
         'UIO-66': [20.7004, 20.7004, 20.7004, 90.0, 90.0, 90.0]}

molpos = {'BEDYEQ': [10, 10, 10, 30, 30, 30],
          'IRMOF-1': [5, 5, 5, 20, 20, 20],
          'UIO-66': [5, 5, 5, 20, 20, 20]}

n_molecules = 1
molecule_padding = 5
packmol_dir = '/home/kutay/Documents/Research/DMMP/packmol'

In [7]:
for mof_name in cells:
    mof_file = os.path.join(packmol_dir, '%s-111.xyz' % mof_name)
    mof = Molecule(read=mof_file)
    
    # Replicate and get box size
    mof.set_cell(cells[mof_name])
    mof.cell.calculate_vertices()   
    box_size = cells[mof_name]
    
    pmol = Packmol()
    pmol.options['tolerance'] = 3.0
    # Add mof
    mof_position = {'fixed': '%.1f %.1f %.1f 0. 0. 0.' % (box_size[0] / 2, box_size[1] / 2, box_size[2] / 2), 'centerofmass': ''}
    pmol.add_structure({'structure': '%s-111.xyz' % mof_name, 'number': '1', 'position': mof_position})
    
    # Add molecule
    mol_file = os.path.join(packmol_dir, 'dmma.xyz')
    mol_position = {'inside box': '%s' % ' '.join([str(round(i, 1)) for i in molpos[mof_name]])}
    pmol.add_structure({'structure': 'dmmp.xyz', 'number': n_molecules, 'position': mol_position})
    
    run_dir = os.path.join(packmol_dir, '%s' % mof_name)
    pmol.run(run_dir, packmol_dir)
    
    # Create cif file
    packed_xyz = Molecule(read=os.path.join(run_dir, pmol.options['output']))
    cell_center = (mof.cell.vertices[0] + mof.cell.vertices[-1]) / 2
    packed_xyz.center(cell_center)
    packed_xyz.write(os.path.join(packmol_dir, '%s-dmmp.cif' % mof_name), cell=mof.cell.to_list())
    
    pmol.clear()

### LAMMPS

In [15]:
from thermof import Simulation
from thermof import Parameters
lammps_dir = '/home/kutay/Documents/Research/DMMP/lammps'

In [16]:
simpar = Parameters()

In [20]:
simpar.lammps

{'bulk_moduli': False,
 'cif_file': '/home/kutay/Documents/Research/DMMP/packmol/BEDYEQ-dmmp.cif',
 'cutoff': 12.5,
 'deposit': 0,
 'dont_replicate_molecules': True,
 'dreid_bond_type': 'harmonic',
 'dump_dcd': 0,
 'dump_lammpstrj': 0,
 'dump_xyz': 0,
 'fix_metal': False,
 'force_field': 'UFF4MOF',
 'h_bonding': False,
 'insert_molecule': '',
 'iter_count': 10,
 'max_dev': 0.01,
 'minimize': False,
 'mol_ff': None,
 'neighbour_size': 5,
 'neqstp': 200000,
 'nprodstp': 200000,
 'npt': False,
 'nvt': False,
 'orthogonalize': False,
 'output_cif': False,
 'output_raspa': False,
 'pressure': 1.0,
 'random_vel': False,
 'replication': '2 2 2',
 'restart': False,
 'temp': 298.0,
 'thermal_scaling': False,
 'tol': 0.4}

In [19]:
for mof_name in cells:
    sim_dir = os.path.join(lammps_dir, mof_name)
    cif_file = os.path.join(packmol_dir, '%s-dmmp.cif' % mof_name)
    
    simpar = Parameters()
    sim = Simulation(mof=os.path.abspath(cif_file), parameters=simpar)
    sim.set_dir(sim_dir)
    simpar.lammps['replication'] = '2 2 2'
    simpar.lammps['force_field'] = 'UFF4MOF'
    simpar.lammps['dont_replicate_molecules'] = True
    simpar.thermof['fix'] = ['MIN', 'NVT', 'NVE']
    simpar.thermof['min']['edif'] = 1e-3
    simpar.thermof['nvt']['steps'] = 500000
    simpar.thermof['nve']['steps'] = 1000000
    simpar.thermof['thermo_style'] = ['step', 'temp', 'press', 'pe', 'etotal', 'emol', 'epair', 'vol', 'lx', 'ly', 'lz']
    simpar.job['nodes'] = 1
    simpar.job['ppn'] = 28
    simpar.job['walltime'] = '36:00:00'
    simpar.job['name'] = '%s-dmmp' % mof_name
    sim.initialize()

Removing existing simulation directory -> /home/kutay/Documents/Research/DMMP/lammps/IRMOF-1
I. Writing Lammps input and data files...
No bonds reported in cif file - computing bonding..
Molecules found in the framework, separating.
Re-sizing to a 2 x 2 x 2 supercell. 
Re-labelling atom indices.
Files created! -> /home/kutay/Documents/Research/DMMP/lammps/IRMOF-1
II. Updating Lammps input file -> /home/kutay/Documents/Research/DMMP/lammps/IRMOF-1/in.IRMOF-1-dmmp
Adding fixes: MIN | NVT | NVE
Updating simulation parameters...
III. Writing slurm job submission file -> /home/kutay/Documents/Research/DMMP/lammps/IRMOF-1/job.IRMOF-1-dmmp
Simulation parameters saved -> /home/kutay/Documents/Research/DMMP/lammps/IRMOF-1/simpar.yaml
Done!
Removing existing simulation directory -> /home/kutay/Documents/Research/DMMP/lammps/UIO-66
I. Writing Lammps input and data files...


/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 0 and 8 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 2 and 9 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 4 and 10 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 6 and 11 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 12 and 18 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_po

No bonds reported in cif file - computing bonding..
Molecules found in the framework, separating.


Re-sizing to a 2 x 2 x 2 supercell. 
Re-labelling atom indices.
Files created! -> /home/kutay/Documents/Research/DMMP/lammps/UIO-66
II. Updating Lammps input file -> /home/kutay/Documents/Research/DMMP/lammps/UIO-66/in.UIO-66-dmmp
Adding fixes: MIN | NVT | NVE
Updating simulation parameters...
III. Writing slurm job submission file -> /home/kutay/Documents/Research/DMMP/lammps/UIO-66/job.UIO-66-dmmp
Simulation parameters saved -> /home/kutay/Documents/Research/DMMP/lammps/UIO-66/simpar.yaml
Done!


/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 2 and 20 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 5 and 21 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 0 and 24 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 3 and 25 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 28 and 560 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled

Removing existing simulation directory -> /home/kutay/Documents/Research/DMMP/lammps/BEDYEQ
I. Writing Lammps input and data files...
No bonds reported in cif file - computing bonding..
Molecules found in the framework, separating.
Re-sizing to a 2 x 2 x 2 supercell. 
Re-labelling atom indices.
Files created! -> /home/kutay/Documents/Research/DMMP/lammps/BEDYEQ
II. Updating Lammps input file -> /home/kutay/Documents/Research/DMMP/lammps/BEDYEQ/in.BEDYEQ-dmmp
Adding fixes: MIN | NVT | NVE
Updating simulation parameters...
III. Writing slurm job submission file -> /home/kutay/Documents/Research/DMMP/lammps/BEDYEQ/job.BEDYEQ-dmmp
Simulation parameters saved -> /home/kutay/Documents/Research/DMMP/lammps/BEDYEQ/simpar.yaml
Done!
